<p><h4>Group of friends are in Bengaluru on a trip, and are looking for brewery in Bengaluru. They are currently located in Jayanagar area. So this program will help them find good brewery with good ratings also with some suggestion for dinner. </h4><p>

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Libraries imported.')

Libraries imported.


In [2]:
CLIENT_ID = 'V1RUF0LB23X4M02JB0ZT050TOHBO4C0AMQTQVVIX1ULWUI2K' # your Foursquare ID
CLIENT_SECRET = 'H41GOW50XHMWN2SXDELGWHALUXEPFM2I1YZ0XT4P3NDSTWV0' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [3]:
address = 'Jayanagar, Bangalore'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

12.9292731 77.5824229


In [4]:
search_query = 'brewery'
radius = 10000

#### Define the corresponding URL

In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=V1RUF0LB23X4M02JB0ZT050TOHBO4C0AMQTQVVIX1ULWUI2K&client_secret=H41GOW50XHMWN2SXDELGWHALUXEPFM2I1YZ0XT4P3NDSTWV0&ll=12.9292731,77.5824229&v=20180604&query=brewery&radius=10000&limit=30'

<p><b>Getting JSON file</b></p>

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d1b77c66adbf5002c5ef11e'},
 'response': {'venues': [{'id': '5c2f66f50fe7a0002c6a20b2',
    'name': 'Levitate Brewery And Kitchen',
    'location': {'lat': 12.907777,
     'lng': 77.599728,
     'labeledLatLngs': [{'label': 'display',
       'lat': 12.907777,
       'lng': 77.599728}],
     'distance': 3041,
     'postalCode': '560076',
     'cc': 'IN',
     'city': 'Bangalore',
     'state': 'Karnātaka',
     'country': 'India',
     'formattedAddress': ['Bangalore 560076', 'Karnātaka', 'India']},
    'categories': [{'id': '50327c8591d4c4b30a586d5d',
      'name': 'Brewery',
      'pluralName': 'Breweries',
      'shortName': 'Brewery',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/brewery_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1562081222',
    'hasPerk': False},
   {'id': '52593f0511d2b20fd9816685',
    'name': 'Vapour - Pub and Brewery',
    'location': {'address': '#773',
     'crossStr

<p><b>Geting relevant part of JSON and transform it into a *pandas* dataframe</b><p>

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",False,5c2f66f50fe7a0002c6a20b2,NaN,IN,Bangalore,India,NaN,3041,"[Bangalore 560076, Karnātaka, India]","[{'label': 'display', 'lat': 12.907777, 'lng':...",12.907777,77.599728,NaN,560076,Karnātaka,Levitate Brewery And Kitchen,v-1562081222,NaN
1,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",False,52593f0511d2b20fd9816685,#773,IN,Bangalore,India,"100ft Rd, Indiranagar",7791,"[#773 (100ft Rd, Indiranagar), Bangalore 56003...","[{'label': 'display', 'lat': 12.96940004248027...",12.969400,77.641268,Indiranagar,560038,Karnātaka,Vapour - Pub and Brewery,v-1562081222,85098408
2,"[{'id': '4eb1bea83b7b6f98df247e06', 'name': 'F...",False,50f6443be4b03f0d25f33282,NaN,IN,NaN,India,NaN,4852,[India],"[{'label': 'display', 'lat': 12.88973296367613...",12.889733,77.563602,NaN,NaN,NaN,Khodays Brewery,v-1562081222,NaN


In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Levitate Brewery And Kitchen,Brewery,NaN,IN,Bangalore,India,NaN,3041,"[Bangalore 560076, Karnātaka, India]","[{'label': 'display', 'lat': 12.907777, 'lng':...",12.907777,77.599728,NaN,560076,Karnātaka,5c2f66f50fe7a0002c6a20b2
1,Vapour - Pub and Brewery,Brewery,#773,IN,Bangalore,India,"100ft Rd, Indiranagar",7791,"[#773 (100ft Rd, Indiranagar), Bangalore 56003...","[{'label': 'display', 'lat': 12.96940004248027...",12.969400,77.641268,Indiranagar,560038,Karnātaka,52593f0511d2b20fd9816685
2,Khodays Brewery,Factory,NaN,IN,NaN,India,NaN,4852,[India],"[{'label': 'display', 'lat': 12.88973296367613...",12.889733,77.563602,NaN,NaN,NaN,50f6443be4b03f0d25f33282


In [9]:
dataframe_filtered.name

0    Levitate Brewery And Kitchen
1        Vapour - Pub and Brewery
2                 Khodays Brewery
Name: name, dtype: object

In [10]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

<p><b>Explore Some Pizza for Dinner</b></p>

In [12]:
venue_id = '4fa862b3e4b0ebff2f749f06' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4fa862b3e4b0ebff2f749f06?client_id=V1RUF0LB23X4M02JB0ZT050TOHBO4C0AMQTQVVIX1ULWUI2K&client_secret=H41GOW50XHMWN2SXDELGWHALUXEPFM2I1YZ0XT4P3NDSTWV0&v=20180604'

#### Send GET request for result

In [14]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'delivery', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4fa862b3e4b0ebff2f749f06',
 'name': "Harry's Italian Pizza Bar",
 'contact': {'phone': '2126081007', 'formattedPhone': '(212) 608-1007'},
 'location': {'address': '225 Murray St',
  'lat': 40.71521779064671,
  'lng': -74.01473940209351,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.71521779064671,
    'lng': -74.01473940209351}],
  'postalCode': '10282',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['225 Murray St',
   'New York, NY 10282',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/harrys-italian-pizza-bar/4fa862b3e4b0ebff2f749f06',
 'categories': [{'id': '4bf58dd8d48988d1ca941735',
   'name': 'Pizza Place',
   'pluralName': 'Pizza Places',
   'shortName': 'Pizza',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
    'suffix': '.png'},
   'primary': True},
  {'id': '4bf58dd8d48988d110941735',
   'name': 'Italian Restaurant',
   'pluralName': 'Italian Restauran

 <h3>Get the venue's overall rating</h3>

In [16]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.1


<h3>In this way it help the group to find good brewery near by with restaurant serving pizza for dinner</h3>